In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 100
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data = pd.read_csv("/content/data.csv")
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from sklearn.metrics import classification_report

from sklearn.model_selection import KFold

from sklearn.model_selection import cross_validate, cross_val_score

from sklearn.svm import SVC

from sklearn import metrics
prediction_feature = [ "radius_mean",  'perimeter_mean', 'area_mean', 'symmetry_mean', 'compactness_mean', 'concave points_mean']

targeted_feature = 'diagnosis'

len(prediction_feature)
X = data[prediction_feature]
y = data.diagnosis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=15)

print(X_train)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
def model_building(model, X_train, X_test, y_train, y_test):
    """

    Model Fitting, Prediction And Other stuff
    return ('score', 'accuracy_score', 'predictions' )
    """

    model.fit(X_train, y_train)
    score = model.score(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(predictions, y_test)

    return (score, accuracy, predictions)
models_list = {
    "LogisticRegression" :  LogisticRegression(),
    "RandomForestClassifier" :  RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=5),
    "DecisionTreeClassifier" :  DecisionTreeClassifier(criterion='entropy', random_state=0),
    "SVC" :  SVC(),
}
print(list(models_list.keys()))
print(list(models_list.values()))
def cm_metrix_graph(cm):

    sns.heatmap(cm,annot=True,fmt="d")
    plt.show()
df_prediction = []
confusion_matrixs = []
df_prediction_cols = [ 'model_name', 'score', 'accuracy_score' , "accuracy_percentage"]

for name, model in zip(list(models_list.keys()), list(models_list.values())):

    (score, accuracy, predictions) = model_building(model, X_train, X_test, y_train, y_test )

    print("\n\nClassification Report of '"+ str(name), "'\n")

    print(classification_report(y_test, predictions))

    df_prediction.append([name, score, accuracy, "{0:.2%}".format(accuracy)])

    confusion_matrixs.append(confusion_matrix(y_test, predictions))


df_pred = pd.DataFrame(df_prediction, columns=df_prediction_cols)
df_pred


     radius_mean  perimeter_mean  area_mean  symmetry_mean  compactness_mean  \
274        17.93          115.20      998.9         0.1538           0.07027   
189        12.30           78.83      463.7         0.1667           0.07253   
158        12.06           76.84      448.6         0.1590           0.05241   
257        15.32          103.20      713.3         0.2398           0.22840   
486        14.64           94.21      666.0         0.1409           0.06698   
..           ...             ...        ...            ...               ...   
85         18.46          121.10     1075.0         0.2132           0.10530   
199        14.45           94.49      642.7         0.1950           0.12060   
156        17.68          117.40      963.7         0.1971           0.16650   
384        13.28           85.79      541.8         0.1617           0.08575   
456        11.63           74.87      415.1         0.1799           0.08574   

     concave points_mean  
274         

,model_name,score,accuracy_score,accuracy_percentage
0,LogisticRegression,0.916010,0.909574,90.96%
1,RandomForestClassifier,0.992126,0.925532,92.55%
2,DecisionTreeClassifier,1.000000,0.909574,90.96%
3,SVC,0.923885,0.914894,91.49%


In [ ]:

def breast_cancer_risk(input_features, base_risk=0.1):

    input_scaled = sc.transform([input_features])

    raw_probability = model.predict_proba(input_scaled)[0][1]

    adjusted_risk = raw_probability * base_risk / 0.5

    risk_percentage = min(adjusted_risk * 100, 100)

    if raw_probability > 0.7:
        risk_percentage *= 1.5

    return min(risk_percentage, 100)

def ask_user_friendly_questions():
    print("Please answer the following questions to estimate your breast cancer risk:")

    age = int(input("What is your age? "))
    if age < 40:
        base_risk = 0.05
    elif 40 <= age < 50:
        base_risk = 0.1
    elif 50 <= age < 60:
        base_risk = 0.15
    else:
        base_risk = 0.2

    lump_size = input("Have you noticed any unusual lumps in your breast? (Small/Medium/Large): ").strip().lower()
    if lump_size == 'small':
        radius_mean = 12.0
    elif lump_size == 'medium':
        radius_mean = 15.0
    else:
        radius_mean = 20.0

    family_history = input("Do you have a family history of breast cancer? (Yes/No): ").strip().lower()
    family_history_factor = 1 if family_history == 'yes' else 0
    if family_history_factor:
        base_risk *= 1.8

    breast_density = input("Has a doctor ever told you that you have dense breast tissue? (Yes/No): ").strip().lower()
    density_factor = 1 if breast_density == 'yes' else 0
    if density_factor:
        base_risk *= 1.4

    hormone_replacement = input("Have you taken hormone replacement therapy for menopause? (Yes/No): ").strip().lower()
    hormone_factor = 1 if hormone_replacement == 'yes' else 0
    if hormone_factor:
        base_risk *= 1.2

    mammogram_abnormality = input("Did your latest mammogram report indicate abnormalities? (Yes/No): ").strip().lower()
    mammogram_factor = 1 if mammogram_abnormality == 'yes' else 0
    if mammogram_factor:
        base_risk *= 2

    perimeter_mean = radius_mean * 2
    area_mean = radius_mean * 100
    symmetry_mean = 0.18 + (0.02 * family_history_factor)
    compactness_mean = 0.1 + (0.05 * density_factor)
    concave_points_mean = 0.1 + (0.1 * hormone_factor)

    return [radius_mean, perimeter_mean, area_mean, symmetry_mean, compactness_mean, concave_points_mean], base_risk

input_features, base_risk = ask_user_friendly_questions()
risk_percentage = breast_cancer_risk(input_features, base_risk)
print(f"Your estimated risk of breast cancer is: {risk_percentage:.2f}%")


Please answer the following questions to estimate your breast cancer risk:
What is your age? 56
Have you noticed any unusual lumps in your breast? (Small/Medium/Large): Small
Do you have a family history of breast cancer? (Yes/No): No
Has a doctor ever told you that you have dense breast tissue? (Yes/No): Yes
Have you taken hormone replacement therapy for menopause? (Yes/No): No
Did your latest mammogram report indicate abnormalities? (Yes/No): No
Your estimated risk of breast cancer is: 25.20%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
